In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed

seed(1)
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

import tensorflow
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor 

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
df=pd.read_excel('Pre-Processed-Data.xlsx')
df.head()

,Dew Point,Wind Speed,Pressure,Pressure Grad
0,4.8,2.47,1029.0,-8
1,8.2,7.42,1021.4,0
2,6.8,6.81,1021.8,11
3,3.6,3.94,1033.7,-1
4,6.1,3.33,1033.4,-11


In [3]:
len(df)

1096

In [4]:
X = df.drop(['Wind Speed'], axis=1)
#Assign the Target column as the output 
Y= df['Wind Speed']

In [5]:
X_norm=(X-X.min())/(X.max()-X.min())
X_norm

,Dew Point,Pressure,Pressure Grad
0,0.487805,0.752055,0.449275
1,0.606272,0.647945,0.565217
2,0.557491,0.653425,0.724638
3,0.445993,0.816438,0.550725
4,0.533101,0.812329,0.405797
...,...,...,...
1091,0.411150,0.706849,0.623188
1092,0.470383,0.767123,0.594203
1093,0.595819,0.795890,0.594203
1094,0.564460,0.831507,0.565217


In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_norm, Y, test_size=0.2, random_state=0)

In [7]:
model = Sequential()
model.add(Dense(32, input_dim=x_train.shape[1], activation="sigmoid", kernel_initializer='normal'))
model.add(Dropout(0.2)) #dropping a few neurons for generalizing the model
model.add(Dense(1, activation="linear", kernel_initializer='normal'))
adam = Adam(learning_rate=1e-3, decay=1e-3)

# Compile model
model.compile(loss="mean_squared_error", optimizer='adam', metrics=['mse','mae'])

In [8]:
print('Fit model...')
filepath="/home/m-marouni/Documents/CE-901/Heathrow/best_weights"
checkpoint = ModelCheckpoint(filepath, monitor='val_mae', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_mae', patience=100, verbose=1, mode='min')
callbacks_list = [checkpoint, early_stopping]

log = model.fit(x_train, y_train,
          validation_split=0.40, batch_size=30, epochs=1000, shuffle=True, callbacks=callbacks_list)

Fit model...
Epoch 1/1000
18/18 [==============================] - 11s 33ms/step - loss: 43.4118 - mse: 43.4118 - mae: 6.0505 - val_loss: 36.3289 - val_mse: 36.3289 - val_mae: 5.5361

Epoch 00001: val_mae improved from inf to 5.53611, saving model to /home/m-marouni/Documents/CE-901/Heathrow/best_weights
INFO:tensorflow:Assets written to: /home/m-marouni/Documents/CE-901/Heathrow/best_weights/assets
Epoch 2/1000
18/18 [==============================] - 0s 6ms/step - loss: 39.1217 - mse: 39.1217 - mae: 5.6812 - val_loss: 32.9844 - val_mse: 32.9844 - val_mae: 5.2252

Epoch 00002: val_mae improved from 5.53611 to 5.22516, saving model to /home/m-marouni/Documents/CE-901/Heathrow/best_weights
INFO:tensorflow:Assets written to: /home/m-marouni/Documents/CE-901/Heathrow/best_weights/assets
Epoch 3/1000
18/18 [==============================] - 0s 5ms/step - loss: 36.5349 - mse: 36.5349 - mae: 5.4084 - val_loss: 29.7890 - val_mse: 29.7890 - val_mae: 4.9096

Epoch 00003: val_mae improved from 5

In [9]:
df_test = pd.read_excel('test-data.xlsx')
df_test.head()

,Wind Speed,Pressure,Pressure Gradient,Dew Point
0,5,1010.8,3,0.8
1,5,1014.4,1,3.1
2,7,1015.7,0,2.6
3,10,1016.5,0,3.3
4,7,1017.1,0,3.4


In [10]:
X2 = df_test.drop(['Wind Speed'], axis=1)
#Assign the Target column as the output 
Y2= df_test['Wind Speed']

In [11]:
X2_norm=(X2-X2.min())/(X2.max()-X2.min())
X2_norm

,Pressure,Pressure Gradient,Dew Point
0,0.459108,0.612903,0.505376
1,0.526022,0.548387,0.629032
2,0.550186,0.516129,0.602151
3,0.565056,0.516129,0.639785
4,0.576208,0.516129,0.645161
...,...,...,...
85,0.657993,0.612903,0.655914
86,0.723048,0.580645,0.849462
87,0.775093,0.483871,0.854839
88,0.765799,0.290323,0.741935


In [12]:
new_predictions=model.predict(X2_norm)

In [13]:
new_predictions

array([[5.614427 ],
       [6.090235 ],
       [6.226926 ],
       [6.2684937],
       [6.2708406],
       [6.4373846],
       [5.8523083],
       [4.8347416],
       [6.0576735],
       [7.41684  ],
       [7.3076744],
       [5.73174  ],
       [6.914847 ],
       [4.2989774],
       [7.715503 ],
       [5.8884788],
       [6.4190216],
       [9.304045 ],
       [9.728127 ],
       [7.745251 ],
       [5.492702 ],
       [6.0173454],
       [6.051252 ],
       [4.247734 ],
       [4.9111385],
       [7.2678084],
       [8.545501 ],
       [7.5993595],
       [7.0960574],
       [6.744134 ],
       [6.588749 ],
       [6.688256 ],
       [6.651318 ],
       [4.841563 ],
       [6.792222 ],
       [7.4039707],
       [7.548912 ],
       [6.156621 ],
       [5.330248 ],
       [4.550278 ],
       [3.9883413],
       [5.138219 ],
       [5.057743 ],
       [6.428398 ],
       [7.5291324],
       [7.9046307],
       [6.5927267],
       [7.9553924],
       [5.843073 ],
       [7.700498 ],


In [14]:
mean_absolute_error(Y2, new_predictions)

3.1659089830186633

In [15]:
mean_squared_error(Y2, new_predictions)

17.650273321534794

In [16]:
df_test['Predicitions'] = new_predictions
#Save the dataframe as csv file

In [17]:
df_test['Predicitions'] = df_test['Predicitions'].apply(lambda x: int(x) if x == x else "")
df_test.to_csv('results102.xlsx', index = False)